### Load Pretrained Model
Load a pretrained Mamba Model that is compatible with Transformers Library

In [ ]:
from modeling_mamba import MambaForCausalLM
from transformers import AutoTokenizer

model = MambaForCausalLM.from_pretrained("Q-bert/Mamba-130M")
tokenizer = AutoTokenizer.from_pretrained("Q-bert/Mamba-130M")

text = "Hi"

input_ids = tokenizer.encode(text, return_tensors="pt")

output = model.generate(input_ids, max_length=20, num_beams=5, no_repeat_ngram_size=2)

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print(generated_text)

In [ ]:
print([(n, type(m)) for n, m in model.named_modules()])

In [ ]:
print(type(model))


def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )


print("plain", print_trainable_parameters(model))

### Add LoRA adapters
1. Identify a particular layer in the Mamba and add an LoRA layer there
2. At this time, is only layer to verify if the code works


In [ ]:
# adapter-1
from peft import LoraConfig, TaskType, get_peft_model

target_modules = ["model.layers.3.x_proj"]

modelA = MambaForCausalLM.from_pretrained("Q-bert/Mamba-130M")
config = LoraConfig(target_modules=target_modules, task_type="CAUSAL_LM")
m1 = get_peft_model(modelA, config)
m1.print_trainable_parameters()
m1.save_pretrained("./wts/ada-1")

In [ ]:
# adapter-2
from peft import LoraConfig, TaskType, get_peft_model

target_modules = ["model.layers.2.x_proj"]
modelB = MambaForCausalLM.from_pretrained("Q-bert/Mamba-130M")

config = LoraConfig(target_modules=target_modules, task_type="CAUSAL_LM")
m2 = get_peft_model(modelB, config)
m2.print_trainable_parameters()
m2.save_pretrained("./wts/ada-2")

### Push them to Hub
push the adapters to hub

In [ ]:
m1.push_to_hub("exp-lora-ada-1", organization="mlsquare")
m2.push_to_hub("exp-lora-ada-2", organization="mlsquare")

In [ ]:
m1.push_to_hub(repo_id="mlsquare/test-1")
m2.push_to_hub(repo_id="mlsquare/test-2")

### Load adapters from the hub

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "Q-bert/Mamba-130M"
peft_model_id_1 = "mlsquare/exp-lora-ada-1"
peft_model_id_2 = "mlsquare/exp-lora-ada-2"

model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True)
print("base mamba", print_trainable_parameters(model))
model.load_adapter(peft_model_id_1, "ada-1")
print("with 1st adapter", print_trainable_parameters(model))
model.load_adapter(peft_model_id_2, "ada-2")
print("with 2nd adapter", print_trainable_parameters(model))

### Merge the adpater into the Model
merge the adapter back to the model, so the merged model will have exactly the same architecture
except with the weights modified

In [ ]:
# https://github.com/huggingface/peft
from peft import PeftMixedModel

model.set_adapter(["ada-1", "ada-2"])
print("base mamba", print_trainable_parameters(model))

output = model.generate(input_ids, max_length=20, num_beams=5, no_repeat_ngram_size=2)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

In [ ]:
# from peft import PeftMixedModel, PeftModel
# model_id = "Q-bert/Mamba-130M"
# base_model = AutoModelForCausalLM.from_pretrained(model_id,trust_remote_code=True)
# peft_model = PeftModel.from_pretrained(base_model, "saddlepoint/exp-lora-ada-1","ada-A")
# peft_model.merge_and_unload()
# print('base mamba',print_trainable_parameters(peft_model))
# peft_model.load_adapter("saddlepoint/exp-lora-ada-2", "ada-2")

In [ ]:
a1 = MambaForCausalLM.from_pretrained("mlsquare/exp-lora-ada-1")
a2 = MambaForCausalLM.from_pretrained("mlsquare/exp-lora-ada-2")

In [ ]:
a1.save_pretrained("./mbins/tmp/ada-1")
a2.save_pretrained("./mbins/tmp/ada-2")

In [ ]:
from peft import PeftMixedModel

base_model = MambaForCausalLM.from_pretrained("Q-bert/Mamba-130M")
print("base mamba", print_trainable_parameters(base_model))
peft_model = PeftMixedModel.from_pretrained(base_model, "./mbins/tmp/ada-1", "adapter1")
print("base mamba", print_trainable_parameters(base_model))
peft_model.load_adapter("./mbins/tmp/ada-2", "adapter2")
peft_model.set_adapter(["adapter1", "adapter2"])
print("base mamba", print_trainable_parameters(base_model))

output = peft_model.generate(
    input_ids, max_length=20, num_beams=5, no_repeat_ngram_size=2
)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

In [ ]:
peft_model.merge_and_unload()
print("merged mamba", print_trainable_parameters(peft_model))
peft_model.base_model.save_pretrained("./mbins/fed-hf/")

In [ ]:
output = peft_model.generate(
    input_ids, max_length=20, num_beams=5, no_repeat_ngram_size=2
)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)